# Library Imports

Import necessary libraries for data processing, visualization, and machine learning.

In [1]:
# Core data handling
import polars as pl
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ML & preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, classification_report
from sklearn.decomposition import PCA
import xgboost as xgb
from lightgbm import LGBMRegressor, LGBMClassifier

# Embeddings|
from sentence_transformers import SentenceTransformer
import torch

# Dimensionality reduction
import umap

# Optional: SQL-style exploration
import duckdb


# Declare Paths


In [2]:
target_path = "C:\\Users\\wongb\\Bridge-ML\\Bridge-ML-LLM-Embedding-Architecture\\data\\targets_10Sep.csv"
NBIfeatures_path = "C:\\Users\\wongb\\Bridge-ML\\Bridge-ML-LLM-Embedding-Architecture\\data\\NBIfull.csv"

# Declare Schema

In [3]:
modified_nbi_schema = {
    "STRUCTURE_ID": {
        "category": "Bridge structure identifier",
        "type": "reference",
        "datatype": float,
        "unit": "decimal degrees",
        "code_map": None
    },
    "LAT": {
        "category": "Bridge latitude",
        "type": "reference",
        "datatype": float,
        "unit": "decimal degrees",
        "code_map": None
    },
    "LONG": {
        "category": "Bridge longitude",
        "type": "reference",
        "datatype": float,
        "unit": "decimal degrees",
        "code_map": None
    },
    "COORDINATES": {
        "category": "Bridge coordinates (latitude, longitude)",
        "type": "reference",
        "datatype": tuple,
        "unit": "decimal degrees",
        "code_map": None
    },
    "STATE_CODE_001": {
        "category": "State code (FIPS)",
        "type": "nominal",
        "datatype": "str",
        "unit": None,
        "code_map": {
            "01": "Alabama",
            "02": "Alaska",
            "04": "Arizona",
            "05": "Arkansas",
            "06": "California",
            "08": "Colorado",
            "09": "Connecticut",
            "10": "Delaware",
            "11": "District of Columbia",
            "12": "Florida",
            "13": "Georgia",
            "15": "Hawaii",
            "16": "Idaho",
            "17": "Illinois",
            "18": "Indiana",
            "19": "Iowa",
            "20": "Kansas",
            "21": "Kentucky",
            "22": "Louisiana",
            "23": "Maine",
            "24": "Maryland",
            "25": "Massachusetts",
            "26": "Michigan",
            "27": "Minnesota",
            "28": "Mississippi",
            "29": "Missouri",
            "30": "Montana",
            "31": "Nebraska",
            "32": "Nevada",
            "33": "New Hampshire",
            "34": "New Jersey",
            "35": "New Mexico",
            "36": "New York",
            "37": "North Carolina",
            "38": "North Dakota",
            "39": "Ohio",
            "40": "Oklahoma",
            "41": "Oregon",
            "42": "Pennsylvania",
            "44": "Rhode Island",
            "45": "South Carolina",
            "46": "South Dakota",
            "47": "Tennessee",
            "48": "Texas",
            "49": "Utah",
            "50": "Vermont",
            "51": "Virginia",
            "53": "Washington",
            "54": "West Virginia",
            "55": "Wisconsin",
            "56": "Wyoming",
            "60": "American Samoa",
            "66": "Guam",
            "69": "Northern Marianas",
            "72": "Puerto Rico",
            "78": "Virgin Islands"
        }
    },
    "COUNTY_CODE_003": {
        "category": "County location code",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "ADAMS",
            "3": "ASOTIN",
            "5": "BENTON",
            "7": "CHELAN",
            "9": "CLALLAM",
            "11": "CLARK",
            "13": "COLUMBIA",
            "15": "COWLITZ",
            "17": "DOUGLAS",
            "19": "FERRY",
            "21": "FRANKLIN",
            "23": "GARFIELD",
            "25": "GRANT",
            "27": "GRAYS HARBOR",
            "29": "ISLAND",
            "31": "JEFFERSON",
            "33": "KING",
            "35": "KITSAP",
            "37": "KITTITAS",
            "39": "KLICKITAT",
            "41": "LEWIS",
            "43": "LINCOLN",
            "45": "MASON",
            "47": "OKANOGAN",
            "49": "PACIFIC",
            "51": "PEND OREILLE",
            "53": "PIERCE",
            "55": "SAN JUAN",
            "57": "SKAGIT",
            "59": "SKAMANIA",
            "61": "SNOHOMISH",
            "63": "SPOKANE",
            "65": "STEVENS",
            "67": "THURSTON",
            "69": "WAHKIAKUM",
            "71": "WALLA WALLA",
            "73": "WHATCOM",
            "75": "WHITMAN",
            "77": "YAKIMA"
        }
    },
    "HIGHWAY_DISTRICT_002": {
        "category": "Highway agency type",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "State Highway Agency",
            "2": "County Highway Agency",
            "3": "Town or Township Highway Agency",
            "4": "City or Municipal Highway Agency"
        }
    },
    "ROUTE_PREFIX_005B": {
        "category": "Route prefix type",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Interstate highway",
            "2": "U.S. numbered highway",
            "3": "State highway",
            "4": "County highway",
            "5": "City street",
            "6": "Federal lands road",
            "7": "State lands road",
            "8": "Other (include toll roads not otherwise indicated or identified above)"
        }
    },
    "SERVICE_LEVEL_005C": {
        "category": "Route service level",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "Other",
            "1": "Mainline",
            "2": "Alternate",
            "3": "Bypass",
            "4": "Spur",
            "6": "Business",
            "7": "Ramp, Wye, Connector, etc.",
            "8": "Service and/or unclassified frontage road"
        }
    },
    "FEATURES_DESC_006A": {
        "category": "Feature intersected description",
        "type": "nl",
        "datatype": str,
        "unit": None,
        "code_map": None
    },
    "FACILITY_CARRIED_007": {
        "category": "Facility carried by structure",
        "type": "nl",
        "datatype": str,
        "unit": None,
        "code_map": None
    },
    "LOCATION_009": {
        "category": "Bridge location description",
        "type": "nl",
        "datatype": str,
        "unit": None,
        "code_map": None
    },
    "MIN_VERT_CLR_010": {
        "category": "Minimum vertical clearance",
        "type": "numerical_coded",
        "datatype": float,
        "unit": None,
        "code_map": {
            "99.99": "No minimum clearance applies"
        }
    },
    "MIN_VERT_CLR_010_SEMANTIC": {
        "category": "Minimum vertical clearance (binned)",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "MIN_VERT_CLR_010",
        "bin_rules": [
            { "category": "open",    "min": 99,  "max": 100 },   # special coded value
            { "category": "low",      "min": None, "max": 4.5 },
            { "category": "moderate", "min": 4.5,  "max": 6 },
            { "category": "high",     "min": 6,    "max": 98 }
        ],
        "code_map": {
            "none": "No minimum vertical clearance recorded",
            "low": "Low vertical clearance (<4.5 m)",
            "moderate": "Moderate vertical clearance (4.5–6 m)",
            "high": "High vertical clearance (>6 m)",
            "open": "Open air vertical clearance (no restriction)"
        }
    },
    "BASE_HWY_NETWORK_012": {
        "category": "Base highway network indicator",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "Inventory route is not on the Base Network",
            "1": "Inventory route is on the Base Network"
        }
    },
    "DETOUR_KILOS_019": {
        "category": "Detour length",
        "type": "numerical",
        "datatype": float,
        "unit": "kilometers",
        "code_map": None
    },
    "DETOUR_KILOS_019_SEMANTIC": {
        "category": "Detour length (binned)",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "DETOUR_KILOS_019",
        "bin_rules": [
            { "category": "very_short", "min": 0,    "max": 5    },
            { "category": "short",      "min": 5,    "max": 20   },
            { "category": "moderate",   "min": 20,   "max": 50   },
            { "category": "long",       "min": 50,   "max": None }
        ],
        "code_map": {
            "none": "No detour information recorded",
            "very_short": "very short detour length (0–5 km)",
            "short": "short detour length (5–20 km)",
            "moderate": "moderate detour length (20–50 km)",
            "long": "long detour length (>50 km)"
        }
    },
    "TOLL_020": {
        "category": "Toll designation type",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Toll bridge – toll paid to use the structure",
            "2": "On toll road – toll paid to the overall facility",
            "3": "On free road – toll-free structure and highway",
            "4": "Interstate toll segment under Secretarial Agreement",
            "5": "Toll bridge under separate Secretarial Agreement"
        }
    },
    "MAINTENANCE_021": {
        "category": "Maintenance responsibility",
        "type": "nominal",
        "datatype": "str",
        "unit": None,
        "code_map": {
            "01": "State Highway Agency",
            "02": "County Highway Agency",
            "03": "Town or Township Highway Agency",
            "04": "City or Municipal Highway Agency",
            "11": "State Park, Forest, or Reservation Agency",
            "12": "Local Park, Forest, or Reservation Agency",
            "21": "Other State Agency",
            "25": "Other Local Agency",
            "26": "Private (non-railroad)",
            "27": "Railroad",
            "31": "State Toll Authority",
            "32": "Local Toll Authority",
            "60": "Other Federal Agency",
            "61": "Indian Tribal Government",
            "62": "Bureau of Indian Affairs",
            "63": "Bureau of Fish and Wildlife",
            "64": "U.S. Forest Service",
            "66": "National Park Service",
            "67": "Tennessee Valley Authority",
            "68": "Bureau of Land Management",
            "69": "Bureau of Reclamation",
            "70": "Corps of Engineers (Civil)",
            "71": "Corps of Engineers (Military)",
            "72": "Air Force",
            "73": "Navy/Marines",
            "74": "Army",
            "75": "NASA",
            "76": "Metropolitan Washington Airports Service",
            "80": "Unknown"
        }
    },

    "FUNCTIONAL_CLASS_026": {
        "category": "Functional classification of route",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Rural - Principal Arterial: Interstate",
            "2": "Rural - Principal Arterial: Other",
            "6": "Rural - Minor Arterial",
            "7": "Rural - Major Collector",
            "8": "Rural - Minor Collector",
            "9": "Rural - Local",
            "11": "Urban - Principal Arterial: Interstate",
            "12": "Urban - Principal Arterial: Other Freeways or Expressways",
            "14": "Urban - Other Principal Arterial",
            "16": "Urban - Minor Arterial",
            "17": "Urban - Collector",
            "19": "Urban - Local"
        }
    },
    "YEAR_BUILT_027": {
        "category": "Year structure was built",
        "type": "temporal",
        "datatype": int,
        "unit": "year",
        "code_map": None
    },
    "YEAR_BUILT_027_DIFF": {
        "category": "Structure age since construction",
        "type": "numerical",
        "datatype": "float",
        "unit": "years",
        "code_map": None
    },
    "YEAR_BUILT_027_SEMANTIC": {
        "category": "Year built",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "YEAR_BUILT_027_DIFF",
        "bin_rules": [
            { "category": "pre_seismic",    "min": None, "max": 1970 },
            { "category": "early_seismic",  "min": 1970, "max": 1990 },
            { "category": "modern_seismic", "min": 1990, "max": 2008 },
            { "category": "high_ductility", "min": 2008, "max": None }
        ],
        "code_map": {
            "none": "Year built not recorded",
            "pre_seismic": "built before modern seismic design standards (pre-1970)",
            "early_seismic": "built during the early seismic design era (1970–1989)",
            "modern_seismic": "built under modern seismic design practices (1990–2007)",
            "high_ductility": "built under contemporary high-ductility seismic codes (2008+)"
        }
    },
    "TRAFFIC_LANES_ON_028A": {
        "category": "Number of traffic lanes on structure",
        "type": "numerical",
        "datatype": float,
        "unit": "lanes",
        "code_map": None
    },
    "TRAFFIC_LANES_ON_028A_SEMANTIC": {
        "category": "Lanes on the structure",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "TRAFFIC_LANES_ON_028A",
        "bin_rules": [
            { "category": "narrow",      "min": 1,    "max": 3    },
            { "category": "standard",    "min": 3,    "max": 5    },
            { "category": "wide",        "min": 5,    "max": 7    },
            { "category": "very_wide",   "min": 7,    "max": None }
        ],
        "code_map": {
            "none": "Number of lanes on the structure not recorded",
            "narrow": "carries a narrow roadway (1–2 lanes)",
            "standard": "carries a standard multi-lane roadway (3–4 lanes)",
            "wide": "carries a wide multi-lane roadway (5–6 lanes)",
            "very_wide": "carries a very wide roadway (7+ lanes)"
        }
    },
    "TRAFFIC_LANES_UND_028B": {
        "category": "Number of traffic lanes under structure",
        "type": "numerical",
        "datatype": float,
        "unit": "lanes",
        "code_map": None
    },
    "TRAFFIC_LANES_UND_028B_SEMANTIC": {
        "category": "Lanes under the structure",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "TRAFFIC_LANES_UND_028B",
        "bin_rules": [
            { "category": "none",             "min": None, "max": 0    },
            { "category": "very_narrow_under","min": 1,    "max": 3    },
            { "category": "narrow_under",     "min": 3,    "max": 5    },
            { "category": "moderate_under",   "min": 5,    "max": 9    },
            { "category": "wide_under",       "min": 9,    "max": 13   },
            { "category": "very_wide_under",  "min": 13,   "max": None }
        ],
        "code_map": {
            "none": "no roadway passes beneath the structure",
            "very_narrow_under": "crosses over a very narrow roadway (1–2 lanes)",
            "narrow_under": "crosses over a narrow roadway (3–4 lanes)",
            "moderate_under": "crosses over a moderate-width roadway (5–8 lanes)",
            "wide_under": "crosses over a wide roadway (9–12 lanes)",
            "very_wide_under": "crosses over a very wide roadway (13+ lanes)"
        }
    },
    "ADT_029": {
        "category": "Average daily traffic volume",
        "type": "numerical",
        "datatype": float,
        "unit": "vehicles per day",
        "code_map": None
    },
    "ADT_029_SEMANTIC": {
        "category": "Average daily traffic",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "ADT_029",
        "bin_rules": [
            { "category": "very_low",   "min": 0,      "max": 5000   },
            { "category": "low",        "min": 5000,   "max": 20000  },
            { "category": "moderate",   "min": 20000,  "max": 50000  },
            { "category": "high",       "min": 50000,  "max": 100000 },
            { "category": "very_high",  "min": 100000, "max": None   }
        ],
        "code_map": {
            "none": "Traffic volume not recorded",
            "very_low": "very low traffic volume (0–5k ADT)",
            "low": "low traffic volume (5k–20k ADT)",
            "moderate": "moderate traffic volume (20k–50k ADT)",
            "high": "high traffic volume (50k–100k ADT)",
            "very_high": "very high traffic volume (>100k ADT)"
        }
    },
    "DESIGN_LOAD_031": {
    "category": "Design load classification",
    "type": "nominal",
    "datatype": int,
    "unit": None,
    "code_map": {
        "0": "Other or unknown",
        "1": "M 9 / H 10",
        "2": "M 13.5 / H 15",
        "3": "MS 13.5 / HS 15",
        "4": "M 18 / H 20",
        "5": "MS 18 / HS 20",
        "6": "MS 18+Mod / HS 20+Mod",
        "7": "Pedestrian",
        "8": "Railroad",
        "9": "MS 22.5 / HS 25"
    }
    },
    "APPR_WIDTH_MT_032": {
        "category": "Approach roadway width",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "APPR_WIDTH_MT_032_SEMANTIC": {
    "category": "Approach roadway width",
    "type": "nominal_derived",
    "datatype": "string",
    "unit": None,
    "source": "APPR_WIDTH_MT_032",
    "bin_rules": [
        { "category": "very_narrow", "min": 0,   "max": 6    },
        { "category": "narrow",      "min": 6,   "max": 10   },
        { "category": "standard",    "min": 10,  "max": 15   },
        { "category": "wide",        "min": 15,  "max": 25   },
        { "category": "very_wide",   "min": 25,  "max": None }
    ],
    "code_map": {
        "none": "Approach roadway width not recorded",
        "very_narrow": "a very narrow approach roadway (0–6 m)",
        "narrow": "a narrow approach roadway (6–10 m)",
        "standard": "a standard-width approach roadway (10–15 m)",
        "wide": "a wide approach roadway (15–25 m)",
        "very_wide": "a very wide approach roadway (over 25 m)"
        }
    },
    "MEDIAN_CODE_033": {
        "category": "Bridge median type",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "No median",
            "1": "Open median",
            "2": "Closed median (no barriers)",
            "3": "Closed median with non-mountable barriers"
        }
    },
    "DEGREES_SKEW_034": {
        "category": "Skew angle of bridge",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "degrees",
        "code_map": {
            "99": "Major variation in skews of substructure units"
        }
    },
    "DEGREES_SKEW_034_SEMANTIC": {
        "category": "Skew angle",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "DEGREES_SKEW_034",
        "bin_rules": [
            { "category": "major",    "min": 99,  "max": 100 },   # special coded value
            { "category": "low",      "min": 0,   "max": 15   },
            { "category": "moderate", "min": 15,  "max": 30   },
            { "category": "high",     "min": 30,  "max": 60   }
        ],
        "code_map": {
            "none": "Skew angle not recorded",
            "low": "low skew angle (0–15°)",
            "moderate": "moderate skew angle (15–30°)",
            "high": "high skew angle (30–60°)",
            "major": "major skew variation across substructure units (coded as 99)"
        }
    },
    "STRUCTURE_FLARED_035": {
        "category": "Structure flared indicator",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "No flare",
            "1": "Yes, flared"
        }
    },
    "RAILINGS_036A": {
        "category": "Bridge railings adequacy",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "0": "Does not meet standards or required but not provided",
            "1": "Meets currently acceptable standards",
            "N": "Not applicable or not required",
        }
    },
    "TRANSITIONS_036B": {
        "category": "Bridge transition adequacy",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "0": "Does not meet standards or required but not provided",
            "1": "Meets currently acceptable standards",
            "N": "Not applicable or not required"
        }
    },
    "APPR_RAIL_036C": {
        "category": "Approach guardrail adequacy",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "0": "Does not meet standards or required but not provided",
            "1": "Meets currently acceptable standards",
            "N": "Not applicable or not required"
        }
    },
    "APPR_RAIL_END_036D": {
        "category": "Approach guardrail end adequacy",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "0": "Does not meet standards or required but not provided",
            "1": "Meets currently acceptable standards",
            "N": "Not applicable or not required"
        }
    },
    "HISTORY_037": {
        "category": "Historical significance",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "On the National Register of Historic Places (NRHP)",
            "2": "Eligible for the NRHP",
            "3": "Possibly eligible or on state/local historic register",
            "4": "Historical significance not determinable",
            "5": "Not eligible for the NRHP"
        }
    },
    "NAVIGATION_038": {
        "category": "Navigation control presence",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "N": "Not applicable, no waterway",
            "0": "No navigation control (permit not required)",
            "1": "Navigation control present (permit required)",
        }
    },
    "NAV_VERT_CLR_MT_039": {
        "category": "Navigation vertical clearance",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "meters",
        "code_map": {
            "0": "No navigation vertical clearance applies"
        }
    },
    "NAV_VERT_CLR_MT_039_SEMANTIC": {
        "category": "Navigation vertical clearance",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "NAV_VERT_CLR_MT_039",
        "default_category": "none",
        "bin_rules": [
            { "category": "none",      "min": None, "max": 0    },
            { "category": "low",       "min": 0,    "max": 10   },
            { "category": "moderate",  "min": 10,   "max": 20   },
            { "category": "high",      "min": 20,   "max": 40   },
            { "category": "very_high", "min": 40,   "max": None }
        ],
        "code_map": {
            "none": "no navigation vertical clearance applies",
            "low": "a low navigation vertical clearance (<10 m)",
            "moderate": "a moderate navigation vertical clearance (10–20 m)",
            "high": "a high navigation vertical clearance (20–40 m)",
            "very_high": "a very high navigation vertical clearance (>40 m)"
        }
    },
    "NAV_HORR_CLR_MT_040": {
        "category": "Navigation horizontal clearance",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "meters",
        "code_map": {
            "0": None,
        }
    },
    "NAV_HORR_CLR_MT_040_SEMANTIC": {
        "category": "Navigation horizontal clearance",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "NAV_HORR_CLR_MT_040",
        "default_category": "none",
        "bin_rules": [
            { "category": "none",      "min": None, "max": 0    },
            { "category": "low",       "min": 0,    "max": 20   },
            { "category": "moderate",  "min": 20,   "max": 40   },
            { "category": "high",      "min": 40,   "max": 60   },
            { "category": "very_high", "min": 60,   "max": None }
        ],
        "code_map": {
            "none": "no navigation horizontal clearance applies",
            "low": "a low navigation horizontal clearance (<20 m)",
            "moderate": "a moderate navigation horizontal clearance (20–40 m)",
            "high": "a high navigation horizontal clearance (40–60 m)",
            "very_high": "a very high navigation horizontal clearance (>60 m)"
        }
    },
    "OPEN_CLOSED_POSTED_041": {
    "category": "Structure open, posted, or closed status",
    "type": "nominal",
    "datatype": str,
    "unit": None,
    "code_map": {
        "A": "Open, no restriction",
        "B": "Open, posting recommended but not legally implemented",
        "D": "Open, would be posted or closed except for temporary shoring, etc.",
        "E": "Open, temporary structure in place while original is closed",
        "G": "New structure not yet open to traffic",
        "K": "Bridge closed to all traffic",
        "P": "Posted for load (may include temporary load restrictions)",
        "R": "Posted for other restrictions (speed, number of vehicles, etc.)",
    }
    },
    "SERVICE_ON_042A": {
        "category": "Type of service on bridge",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Highway",
            "2": "Railroad",
            "3": "Pedestrian-bicycle",
            "4": "Highway-railroad",
            "5": "Highway-pedestrian",
            "6": "Overpass at interchange or 2nd level",
            "7": "Third level interchange",
            "8": "Fourth level interchange",
            "9": "Building or plaza",
            "0": "Other"
        }
    },
    "SERVICE_UND_042B": {
        "category": "Type of service under bridge",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Highway (with or without pedestrian)",
            "2": "Railroad",
            "3": "Pedestrian-bicycle",
            "4": "Highway-railroad",
            "5": "Waterway",
            "6": "Highway-waterway",
            "7": "Railroad-waterway",
            "8": "Highway-waterway-railroad",
            "9": "Relief for waterway",
            "0": "Other"
        }
    },
    "STRUCTURE_KIND_043A": {
        "category": "Kind of material (main structure)",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Concrete",
            "2": "Concrete continuous",
            "3": "Steel",
            "4": "Steel continuous",
            "5": "Prestressed concrete",
            "6": "Prestressed concrete continuous",
            "7": "Wood or timber",
            "8": "Masonry",
            "9": "Aluminum, wrought iron, or cast iron",
            "0": "Other"
        }
    },
    "STRUCTURE_TYPE_043B": {
        "category": "Type of design (main structure)",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Slab",
            "2": "Stringer/Multi-beam or girder",
            "3": "Girder and floorbeam system",
            "4": "Tee beam",
            "5": "Box beams or girders - multiple",
            "6": "Box beams or girders - single or spread",
            "7": "Frame",
            "8": "Orthotropic",
            "9": "Truss - Deck",
            "10": "Truss - Thru",
            "11": "Arch - Deck",
            "12": "Arch - Thru",
            "13": "Suspension",
            "14": "Stayed girder",
            "15": "Movable - Lift",
            "16": "Movable - Bascule",
            "17": "Movable - Swing",
            "18": "Tunnel",
            "19": "Culvert",
            "20": "Mixed types",
            "21": "Segmental box girder",
            "22": "Channel beam",
            "0": "Other"
        }
    },
    "APPR_KIND_044A": {
        "category": "Kind of material (approach spans)",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Concrete",
            "2": "Concrete continuous",
            "3": "Steel",
            "4": "Steel continuous",
            "5": "Prestressed concrete",
            "6": "Prestressed concrete continuous",
            "7": "Wood or timber",
            "8": "Masonry",
            "9": "Aluminum, wrought iron, or cast iron",
            "0": "Other"
        }
    },
    "APPR_TYPE_044B": {
        "category": "Type of design (approach spans)",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Slab",
            "2": "Stringer/Multi-beam or girder",
            "3": "Girder and floorbeam system",
            "4": "Tee beam",
            "5": "Box beams or girders - multiple",
            "6": "Box beams or girders - single or spread",
            "7": "Frame",
            "8": "Orthotropic",
            "9": "Truss - Deck",
            "10": "Truss - Thru",
            "11": "Arch - Deck",
            "12": "Arch - Thru",
            "13": "Suspension",
            "14": "Stayed girder",
            "15": "Movable - Lift",
            "16": "Movable - Bascule",
            "17": "Movable - Swing",
            "18": "Tunnel",
            "19": "Culvert",
            "20": "Mixed types",
            "21": "Segmental box girder",
            "22": "Channel beam",
            "0": "Other"
        }
    },
    "MAIN_UNIT_SPANS_045": {
        "category": "Number of spans in main unit",
        "type": "numerical",
        "datatype": float,
        "unit": "spans",
        "code_map": None
    },
    "MAIN_UNIT_SPANS_045_SEMANTIC": {
        "category": "Number of spans in main unit",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "MAIN_UNIT_SPANS_045",
        "bin_rules": [
            { "category": "none",       "min": None, "max": 1    },
            { "category": "single",     "min": 1,    "max": 2    },
            { "category": "few",        "min": 2,    "max": 5    },
            { "category": "multiple",   "min": 5,    "max": 10   },
            { "category": "many",       "min": 10,   "max": None }
        ],
        "code_map": {
            "none": "no defined main unit spans",
            "single": "a single-span main unit",
            "few": "a few-span main unit (2–4 spans)",
            "multiple": "a multi-span main unit (5–9 spans)",
            "many": "a large-span-count main unit (10+ spans)"
        }
    },
    "APPR_SPANS_046": {
        "category": "Number of approach spans",
        "type": "numerical",
        "datatype": float,
        "unit": "spans",
        "code_map": None
    },
    "APPR_SPANS_046_SEMANTIC": {
        "category": "Number of approach spans",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "APPR_SPANS_046",
        "bin_rules": [
            { "category": "none",       "min": None, "max": 1    },
            { "category": "few",        "min": 1,    "max": 4    },
            { "category": "several",    "min": 4,    "max": 10   },
            { "category": "many",       "min": 10,   "max": None }
        ],
        "code_map": {
            "none": "no approach spans",
            "few": "a few approach spans (1–3 spans)",
            "several": "several approach spans (4–9 spans)",
            "many": "many approach spans (10+ spans)"
        }
    },
    "HORR_CLR_MT_047": {
        "category": "Total horizontal clearance",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "meters",
        "code_map": {
            "999": "Restriction 100 meters or greater"
        }
    },
    "HORR_CLR_MT_047_SEMANTIC": {
        "category": "Total horizontal clearance",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "HORR_CLR_MT_047",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "narrow",      "min": 1,    "max": 10    },
            { "category": "moderate",    "min": 10,   "max": 20    },
            { "category": "wide",        "min": 20,   "max": 40    },
            { "category": "very_wide",   "min": 40,   "max": 100   },
            { "category": "extreme",     "min": 100,  "max": None  }  # includes 999
        ],
        "code_map": {
            "none": "no horizontal clearance specified",
            "narrow": "a narrow horizontal clearance (1–10 m)",
            "moderate": "a moderate horizontal clearance (10–20 m)",
            "wide": "a wide horizontal clearance (20–40 m)",
            "very_wide": "a very wide horizontal clearance (40–100 m)",
            "extreme": "an extremely large horizontal clearance (100+ m)"
        }
    },

    "MAX_SPAN_LEN_MT_048": {
        "category": "Maximum span length",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "MAX_SPAN_LEN_MT_048_SEMANTIC": {
        "category": "Maximum span length",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "MAX_SPAN_LEN_MT_048",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "short",       "min": 1,    "max": 20    },
            { "category": "moderate",    "min": 20,   "max": 50    },
            { "category": "long",        "min": 50,   "max": 100   },
            { "category": "very_long",   "min": 100,  "max": None  }
        ],
        "code_map": {
            "none": "no defined maximum span length",
            "short": "a short maximum span (<20 m)",
            "moderate": "a moderate maximum span (20–50 m)",
            "long": "a long maximum span (50–100 m)",
            "very_long": "a very long maximum span (>100 m)"
        }
    },
    "STRUCTURE_LEN_MT_049": {
        "category": "Structure total length",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "STRUCTURE_LEN_MT_049_SEMANTIC": {
        "category": "Structure length",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "STRUCTURE_LEN_MT_049",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "short",       "min": 1,    "max": 50    },
            { "category": "medium",      "min": 50,   "max": 100   },
            { "category": "long",        "min": 100,  "max": 300   },
            { "category": "very_long",   "min": 300,  "max": None  }
        ],
        "code_map": {
            "none": "no defined structure length",
            "short": "a short structure (1–50 m)",
            "medium": "a medium-length structure (50–100 m)",
            "long": "a long structure (100–300 m)",
            "very_long": "a very long structure (>300 m)"
        }
    },
    "LEFT_CURB_MT_050A": {
        "category": "Left curb or sidewalk width",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "LEFT_CURB_MT_050A_SEMANTIC": {
        "category": "Left curb or sidewalk width",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "LEFT_CURB_MT_050A",
        "bin_rules": [
            { "category": "none",     "min": None, "max": 0     },
            { "category": "narrow",   "min": 0,    "max": 1.5   },
            { "category": "standard", "min": 1.5,  "max": 3.0   },
            { "category": "wide",     "min": 3.0,  "max": None  }
        ],
        "code_map": {
            "none": "no left curb or sidewalk",
            "narrow": "a narrow left curb/sidewalk (0–1.5 m)",
            "standard": "a standard left curb/sidewalk (1.5–3 m)",
            "wide": "a wide left curb/sidewalk (>3 m)"
        }
    },
    "RIGHT_CURB_MT_050B": {
        "category": "Right curb or sidewalk width",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "RIGHT_CURB_MT_050B_SEMANTIC": {
        "category": "Right curb or sidewalk width",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "RIGHT_CURB_MT_050B",
        "bin_rules": [
            { "category": "none",     "min": None, "max": 0     },
            { "category": "narrow",   "min": 0,    "max": 1.5   },
            { "category": "standard", "min": 1.5,  "max": 3.0   },
            { "category": "wide",     "min": 3.0,  "max": None  }
        ],
        "code_map": {
            "none": "no right curb or sidewalk",
            "narrow": "a narrow right curb/sidewalk (0–1.5 m)",
            "standard": "a standard right curb/sidewalk (1.5–3 m)",
            "wide": "a wide right curb/sidewalk (>3 m)"
        }
    },
    "ROADWAY_WIDTH_MT_051": {
        "category": "Bridge roadway width (curb-to-curb)",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "ROADWAY_WIDTH_MT_051_SEMANTIC": {
        "category": "Bridge roadway width",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "ROADWAY_WIDTH_MT_051",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "very_narrow", "min": 1,    "max": 6     },
            { "category": "narrow",      "min": 6,    "max": 10    },
            { "category": "standard",    "min": 10,   "max": 15    },
            { "category": "wide",        "min": 15,   "max": 25    },
            { "category": "very_wide",   "min": 25,   "max": None  }
        ],
        "code_map": {
            "none": "no defined roadway width",
            "very_narrow": "a very narrow roadway width (1–6 m)",
            "narrow": "a narrow roadway width (6–10 m)",
            "standard": "a standard roadway width (10–15 m)",
            "wide": "a wide roadway width (15–25 m)",
            "very_wide": "a very wide roadway width (>25 m)"
        }
    },
    "DECK_WIDTH_MT_052": {
        "category": "Deck width (out-to-out)",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "DECK_WIDTH_MT_052_SEMANTIC": {
        "category": "Deck width",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "DECK_WIDTH_MT_052",
        "bin_rules": [
            { "category": "none",       "min": None, "max": 1     },
            { "category": "narrow",     "min": 1,    "max": 10    },
            { "category": "moderate",   "min": 10,   "max": 20    },
            { "category": "wide",       "min": 20,   "max": 30    },
            { "category": "very_wide",  "min": 30,   "max": None  }
        ],
        "code_map": {
            "none": "no defined deck width",
            "narrow": "a narrow deck width (1–10 m)",
            "moderate": "a moderate deck width (10–20 m)",
            "wide": "a wide deck width (20–30 m)",
            "very_wide": "a very wide deck width (>30 m)"
        }
    },
    "VERT_CLR_OVER_MT_053": {
        "category": "Minimum vertical clearance over roadway",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "meters",
        "code_map": {
            "99.99": "No vertical clearance restriction"
        }
    },
    "VERT_CLR_OVER_MT_053_SEMANTIC": {
        "category": "Vertical clearance over bridge roadway",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "VERT_CLR_OVER_MT_053",
        "bin_rules": [
            { "category": "none",      "min": None, "max": 1       },   # includes 0 and invalid
            { "category": "low",       "min": 1,    "max": 4.5     },
            { "category": "moderate",  "min": 4.5,  "max": 6       },
            { "category": "high",      "min": 6,    "max": 10      },
            { "category": "very_high", "min": 10,   "max": None    }
        ],
        "code_map": {
            "none": "no vertical clearance restriction",
            "low": "a low vertical clearance (<4.5 m)",
            "moderate": "a moderate vertical clearance (4.5–6 m)",
            "high": "a high vertical clearance (6–10 m)",
            "very_high": "a very high vertical clearance (>10 m)"
        }
    },
    "VERT_CLR_UND_REF_054A": {
        "category": "Vertical underclearance reference feature",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "H": "Highway beneath structure",
            "R": "Railroad beneath structure",
            "N": "Feature not a highway or railroad"
        }
    },
    "VERT_CLR_UND_054B": {
        "category": "Minimum vertical underclearance",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "VERT_CLR_UND_054B_SEMANTIC": {
        "category": "Minimum vertical underclearance",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "VERT_CLR_UND_054B",
        "bin_rules": [
            { "category": "none",       "min": None, "max": 1     },
            { "category": "very_low",   "min": 1,    "max": 3     },
            { "category": "low",        "min": 3,    "max": 5     },
            { "category": "moderate",   "min": 5,    "max": 7     },
            { "category": "high",       "min": 7,    "max": 12    },
            { "category": "very_high",  "min": 12,   "max": None  }
        ],
        "code_map": {
            "none": "no vertical underclearance",
            "very_low": "a very low vertical underclearance (<3 m)",
            "low": "a low vertical underclearance (3–5 m)",
            "moderate": "a moderate vertical underclearance (5–7 m)",
            "high": "a high vertical underclearance (7–12 m)",
            "very_high": "a very high vertical underclearance (>12 m)"
        }
    },
    "LAT_UND_REF_055A": {
        "category": "Lateral underclearance reference feature",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "H": "Highway beneath structure",
            "R": "Railroad beneath structure",
            "N": "Feature not a highway or railroad"
        }
    },
    "LAT_UND_MT_055B": {
        "category": "Minimum lateral underclearance (right)",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "LAT_UND_MT_055B_SEMANTIC": {
        "category": "Minimum lateral underclearance (right)",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "LAT_UND_MT_055B",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "very_tight",  "min": 1,    "max": 2     },
            { "category": "tight",       "min": 2,    "max": 4     },
            { "category": "moderate",    "min": 4,    "max": 8     },
            { "category": "wide",        "min": 8,    "max": 12    },
            { "category": "very_wide",   "min": 12,   "max": None  }
        ],
        "code_map": {
            "none": "no lateral underclearance on the right",
            "very_tight": "a very tight right lateral underclearance (<2 m)",
            "tight": "a tight right lateral underclearance (2–4 m)",
            "moderate": "a moderate right lateral underclearance (4–8 m)",
            "wide": "a wide right lateral underclearance (8–12 m)",
            "very_wide": "a very wide right lateral underclearance (>12 m)"
        }
    },
    "LEFT_LAT_UND_MT_056": {
        "category": "Minimum lateral underclearance (left)",
        "type": "numerical",
        "datatype": float,
        "unit": "meters",
        "code_map": None
    },
    "LEFT_LAT_UND_MT_056_SEMANTIC": {
        "category": "Minimum lateral underclearance (left)",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "LEFT_LAT_UND_MT_056",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "very_tight",  "min": 1,    "max": 2     },
            { "category": "tight",       "min": 2,    "max": 4     },
            { "category": "moderate",    "min": 4,    "max": 8     },
            { "category": "wide",        "min": 8,    "max": 12    },
            { "category": "very_wide",   "min": 12,   "max": None  }
        ],
        "code_map": {
            "none": "no lateral underclearance on the left",
            "very_tight": "a very tight left lateral underclearance (<2 m)",
            "tight": "a tight left lateral underclearance (2–4 m)",
            "moderate": "a moderate left lateral underclearance (4–8 m)",
            "wide": "a wide left lateral underclearance (8–12 m)",
            "very_wide": "a very wide left lateral underclearance (>12 m)"
        }
    },
    "DECK_COND_058": {
        "category": "Deck condition rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "Excellent condition",
            "8": "Very good condition – no problems noted",
            "7": "Good condition – some minor problems",
            "6": "Satisfactory condition – minor deterioration",
            "5": "Fair condition – minor section loss, cracking, or scour",
            "4": "Poor condition – advanced section loss or deterioration",
            "3": "Serious condition – loss of section or fatigue/shear cracks",
            "2": "Critical condition – advanced deterioration or scour",
            "1": "Imminent failure – closed, but repair may restore light service",
            "0": "Failed condition – out of service; beyond corrective action"
        }
    },
    "SUPERSTRUCTURE_COND_059": {
        "category": "Superstructure condition rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "Excellent condition",
            "8": "Very good condition – no problems noted",
            "7": "Good condition – some minor problems",
            "6": "Satisfactory condition – minor deterioration",
            "5": "Fair condition – minor section loss, cracking, or scour",
            "4": "Poor condition – advanced section loss or deterioration",
            "3": "Serious condition – loss of section or fatigue/shear cracks",
            "2": "Critical condition – advanced deterioration or scour",
            "1": "Imminent failure – closed, but repair may restore light service",
            "0": "Failed condition – out of service; beyond corrective action",
        }
    },
    "SUBSTRUCTURE_COND_060": {
        "category": "Substructure condition rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "Excellent condition",
            "8": "Very good condition – no problems noted",
            "7": "Good condition – some minor problems",
            "6": "Satisfactory condition – minor deterioration",
            "5": "Fair condition – minor section loss, cracking, or scour",
            "4": "Poor condition – advanced section loss or deterioration",
            "3": "Serious condition – loss of section or fatigue/shear cracks",
            "2": "Critical condition – advanced deterioration or scour",
            "1": "Imminent failure – closed, but repair may restore light service",
            "0": "Failed condition – out of service; beyond corrective action"
        }
    },
    "CHANNEL_COND_061": {
        "category": "Channel and channel protection condition",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "No deficiencies affecting channel condition",
            "8": "Banks protected or well vegetated; stable control devices",
            "7": "Minor bank or control device repairs needed; minor drift",
            "6": "Minor slumping, debris slightly restricting channel",
            "5": "Erosion of bank protection, trees/brush restricting flow",
            "4": "Severely undermined protection; heavy debris deposits",
            "3": "Bank protection failed; channel threatens bridge",
            "2": "Channel change threatens bridge collapse",
            "1": "Bridge closed due to channel failure; repair possible",
            "0": "Bridge closed due to channel failure; replacement necessary"
        }
    },
    "OPR_RATING_METH_063": {
        "category": "Operating rating determination method",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Load Factor (LF)",
            "2": "Allowable Stress (AS)",
            "3": "Load and Resistance Factor (LRFR)",
            "4": "Load Testing",
            "5": "No rating analysis performed"
        }
    },
    "OPERATING_RATING_064": {
        "category": "Operating rating capacity",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "metric tons",
        "code_map": {
            "0": "Bridge will not carry minimum 2.7 metric tons"
        }
    },
    "OPERATING_RATING_064_SEMANTIC": {
        "category": "Operating rating (metric tons)",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "OPERATING_RATING_064",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 2.7   },
            { "category": "very_low",    "min": 2.7,  "max": 10    },
            { "category": "low",         "min": 10,   "max": 20    },
            { "category": "moderate",    "min": 20,   "max": 40    },
            { "category": "high",        "min": 40,   "max": 60    },
            { "category": "very_high",   "min": 60,   "max": None  }
        ],
        "code_map": {
            "none": "no usable operating rating",
            "very_low": "a very low operating rating (<10 metric tons)",
            "low": "a low operating rating (10–20 metric tons)",
            "moderate": "a moderate operating rating (20–40 metric tons)",
            "high": "a high operating rating (40–60 metric tons)",
            "very_high": "a very high operating rating (>60 metric tons)"
        }
    },
    "INV_RATING_METH_065": {
        "category": "Inventory rating determination method",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Load Factor (LF)",
            "2": "Allowable Stress (AS)",
            "3": "Load and Resistance Factor (LRFR)",
            "4": "Load Testing",
            "5": "No rating analysis performed"
        }
    },
    "INVENTORY_RATING_066": {
        "category": "Inventory rating capacity",
        "type": "numerical_coded",
        "datatype": float,
        "unit": "metric tons",
        "code_map": {
            "999": "Structure under sufficient fill; live load insignificant"
        }
    },
    "INVENTORY_RATING_066_SEMANTIC": {
        "category": "Inventory rating (metric tons)",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "INVENTORY_RATING_066",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 2.7   },
            { "category": "very_low",    "min": 2.7,  "max": 10    },
            { "category": "low",         "min": 10,   "max": 20    },
            { "category": "moderate",    "min": 20,   "max": 40    },
            { "category": "high",        "min": 40,   "max": 60    },
            { "category": "very_high",   "min": 60,   "max": None  }
        ],
        "code_map": {
            "none": "no usable inventory rating",
            "very_low": "a very low inventory rating (<10 metric tons)",
            "low": "a low inventory rating (10–20 metric tons)",
            "moderate": "a moderate inventory rating (20–40 metric tons)",
            "high": "a high inventory rating (40–60 metric tons)",
            "very_high": "a very high inventory rating (>60 metric tons)"
        }
    },
    "STRUCTURAL_EVAL_067": {
        "category": "Structural evaluation rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "Superior to present desirable criteria",
            "8": "Equal to present desirable criteria",
            "7": "Better than minimum criteria",
            "6": "Equal to present minimum criteria",
            "5": "Somewhat better than minimum tolerable adequacy",
            "4": "Meets minimum tolerable limits to remain in place",
            "3": "Basically intolerable; high priority corrective action needed",
            "2": "Basically intolerable; high priority replacement needed",
            "1": "Code not used",
            "0": "Bridge closed"
        }
    },
    "DECK_GEOMETRY_EVAL_068": {
        "category": "Deck geometry evaluation rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "Superior to present desirable criteria",
            "8": "Equal to present desirable criteria",
            "7": "Better than minimum criteria",
            "6": "Equal to present minimum criteria",
            "5": "Somewhat better than minimum tolerable adequacy",
            "4": "Meets minimum tolerable limits to remain in place",
            "3": "Basically intolerable; high priority corrective action needed",
            "2": "Basically intolerable; high priority replacement needed",
            "1": "Code not used",
            "0": "Bridge closed"
        }
    },
    "UNDCLRENCE_EVAL_069": {
        "category": "Underclearance evaluation rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "9": "Superior to present desirable criteria",
            "8": "Equal to present desirable criteria",
            "7": "Better than minimum criteria",
            "6": "Equal to present minimum criteria",
            "5": "Somewhat better than minimum tolerable adequacy",
            "4": "Meets minimum tolerable limits to remain in place",
            "3": "Basically intolerable; high priority corrective action needed",
            "2": "Basically intolerable; high priority replacement needed",
            "1": "Code not used",
            "0": "Bridge closed"
        }
    },
    "POSTING_EVAL_070": {
        "category": "Bridge posting evaluation rating",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "5": "Equal to or above legal loads",
            "4": "0.1–9.9% below legal loads",
            "3": "10.0–19.9% below legal loads",
            "2": "20.0–29.9% below legal loads",
            "1": "30.0–39.9% below legal loads",
            "0": ">39.9% below legal loads"
        }
    },
    "WATERWAY_EVAL_071": {
        "category": "Waterway adequacy evaluation rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "N": "Not applicable",
            "9": "Superior to present desirable criteria",
            "8": "Equal to present desirable criteria",
            "7": "Better than minimum criteria",
            "6": "Equal to present minimum criteria",
            "5": "Somewhat better than minimum tolerable adequacy",
            "4": "Meets minimum tolerable limits to remain in place",
            "3": "Basically intolerable; high priority corrective action needed",
            "2": "Basically intolerable; high priority replacement needed",
            "1": "Code not used",
            "0": "Bridge closed"
        }
    },
    "APPR_ROAD_EVAL_072": {
        "category": "Approach roadway alignment evaluation rating",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "N": "Not applicable",
            "9": "Superior to present desirable criteria",
            "8": "Equal to present desirable criteria",
            "7": "Better than minimum criteria",
            "6": "Equal to present minimum criteria",
            "5": "Somewhat better than minimum tolerable adequacy",
            "4": "Meets minimum tolerable limits to remain in place",
            "3": "Basically intolerable; high priority corrective action needed",
            "2": "Basically intolerable; high priority replacement needed",
            "1": "Code not used",
            "0": "Bridge closed"
        }
    },
    "WORK_PROPOSED_075A": {
        "category": "Type of proposed work",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "31": "Replacement due to substandard load capacity or roadway geometry",
            "32": "Replacement due to relocation of road",
            "33": "Widening without deck rehabilitation/replacement (includes culvert lengthening)",
            "34": "Widening with deck rehabilitation or replacement",
            "35": "Bridge rehabilitation due to deterioration or inadequate strength",
            "36": "Deck rehabilitation with incidental widening",
            "37": "Deck replacement with incidental widening",
            "38": "Other structural or hydraulic work"
        }
    },
    "DATE_OF_INSPECT_090": {
        "category": "Date of last routine inspection",
        "type": "temporal",
        "datatype": str,
        "unit": "MMYYYY",
        "code_map": None
    },
    "DATE_OF_INSPECT_090_DIFF": {
        "category": "Years since last routine inspection",
        "type": "numerical",
        "datatype": "float",
        "unit": "years",
        "code_map": None
    },
    "DATE_OF_INSPECT_090_SEMANTIC": {
        "category": "Years since last inspection date",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "DATE_OF_INSPECT_090_DIFF",
        "bin_rules": [
            { "category": "none",          "max_age_years": 0 }, 
            { "category": "recent",        "max_age_years": 2 },
            { "category": "moderately_old","max_age_years": 5 },
            { "category": "old",           "max_age_years": 10 },
            { "category": "very_old",      "max_age_years": None }
        ],
        "code_map": {
            "none": "no recorded inspection date",
            "recent": "inspected within the last 2 years",
            "moderately_old": "inspected 2–5 years ago",
            "old": "inspected 5–10 years ago",
            "very_old": "inspected over 10 years ago"
        }
    },
    "INSPECT_FREQ_MONTHS_091": {
        "category": "Inspection frequency",
        "type": "numerical",
        "datatype": int,
        "unit": "months",
        "code_map": None
    },
    "INSPECT_FREQ_MONTHS_091_SEMANTIC": {
        "category": "Inspection frequency",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "INSPECT_FREQ_MONTHS_091",
        "bin_rules": [
            { "category": "none",            "min": None, "max": 1   },
            { "category": "high_risk",       "min": 1,    "max": 13  },
            { "category": "moderate_risk",   "min": 13,   "max": 25  },
            { "category": "standard",        "min": 25,   "max": 49  },
            { "category": "low_risk",        "min": 49,   "max": None }
        ],
        "code_map": {
            "none": "no designated inspection frequency",
            "high_risk": "inspected frequently (≤12 months)",
            "moderate_risk": "inspected every 12–24 months",
            "standard": "standard inspection frequency (24–48 months)",
            "low_risk": "inspected infrequently (>48 months)"
        }
    },
    "FRACTURE_092A": {
        "category": "Fracture critical inspection requirement",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "Y": "Special inspection or emphasis needed; includes frequency in months (e.g., Y24)",
            "N": "Special inspection not needed"
        }
    },
    "UNDWATER_LOOK_SEE_092B": {
        "category": "Underwater inspection requirement",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "Y": "Underwater inspection or emphasis needed; includes frequency in months (e.g., Y24)",
            "N": "Underwater inspection not needed"
        }
    },
    "SPEC_INSPECT_092C": {
        "category": "Special inspection requirement",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "Y": "Special or other inspection required; includes frequency in months (e.g., Y24)",
            "N": "Special inspection not needed"
        }
    },
    "STRAHNET_HIGHWAY_100": {
        "category": "STRAHNET highway designation",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Not on a STRAHNET route",
            "2": "On an Interstate STRAHNET route",
            "3": "On a Non-Interstate STRAHNET route",
            "4": "On a STRAHNET connector route"
        }
    },
    "PARALLEL_STRUCTURE_101": {
        "category": "Parallel structure designation",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "R": "Right structure of parallel bridges (inventory direction)",
            "L": "Left structure of parallel bridges (opposite direction)",
            "N": "No parallel structure exists"
        }
    },
    "TRAFFIC_DIRECTION_102": {
        "category": "Traffic direction type",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "Highway traffic not carried",
            "1": "One-way traffic",
            "2": "Two-way traffic",
            "3": "One-lane bridge with two-way traffic"
        }
    },
    "HIGHWAY_SYSTEM_104": {
        "category": "Highway system designation",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "Not on National Highway System (NHS)",
            "1": "On National Highway System (NHS)"
        }
    },
    "FEDERAL_LANDS_105": {
        "category": "Federal lands highway designation",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "0": "Not applicable",
            "1": "Indian Reservation Road (IRR)",
            "2": "Forest Highway (FH)",
            "3": "Land Management Highway System (LMHS)",
            "4": "Both IRR and FH",
            "5": "Both IRR and LMHS",
            "6": "Both FH and LMHS",
            "9": "Combined IRR, FH, and LMHS"
        }
    },
    "YEAR_RECONSTRUCTED_106": {
        "category": "Year structure reconstructed",
        "type": "temporal",
        "datatype": int,
        "unit": "year",
        "code_map": {
            "0": "No reconstruction performed"
        }
    },
    "YEAR_RECONSTRUCTED_106_DIFF": {
        "category": "Years since last reconstruction",
        "type": "numerical",
        "datatype": "float",
        "unit": "years",
        "code_map": None
    },
    "YEAR_RECONSTRUCTED_106_SEMANTIC": {
        "category": "Reconstruction age",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "YEAR_RECONSTRUCTED_106_DIFF",
        "bin_rules": [
            { "category": "none",       "min": None, "max": 0    },
            { "category": "recent",     "min": 0,    "max": 10   },
            { "category": "moderate",   "min": 10,   "max": 30   },
            { "category": "old",        "min": 30,   "max": 60   },
            { "category": "very_old",   "min": 60,   "max": None }
        ],
        "code_map": {
            "none": "no reconstruction performed",
            "recent": "reconstructed recently (within 10 years)",

            "moderate": "reconstructed 10–30 years ago",
            "old": "reconstructed 30–60 years ago",
            "very_old": "reconstructed over 60 years ago"
        }
    },
    "DECK_STRUCTURE_TYPE_107": {
        "category": "Deck structure type",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "1": "Concrete cast-in-place",
            "2": "Concrete precast panels",
            "3": "Open grating",
            "4": "Closed grating",
            "5": "Steel plate (includes orthotropic)",
            "6": "Corrugated steel",
            "7": "Aluminum",
            "8": "Wood or timber",
            "9": "Other",
            "0": "None",
            "N": "Not applicable"
        }
    },
    "SURFACE_TYPE_108A": {
        "category": "Deck wearing surface type",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "1": "Monolithic concrete (placed with structural deck)",
            "2": "Integral concrete (non-modified separate layer)",
            "3": "Latex concrete or similar additive",
            "4": "Low slump concrete",
            "5": "Epoxy overlay",
            "6": "Bituminous",
            "7": "Wood or timber",
            "8": "Gravel",
            "9": "Other",
            "0": "None (no added surface)",
            "N": "Not applicable (no deck)"
        }
    },
    "MEMBRANE_TYPE_108B": {
        "category": "Deck membrane type",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "1": "Built-up",
            "2": "Preformed fabric",
            "3": "Epoxy",
            "8": "Unknown",
            "9": "Other",
            "0": "None",
            "N": "Not applicable (no deck)"
        }
    },
    "DECK_PROTECTION_108C": {
        "category": "Deck protection system",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "1": "Epoxy-coated reinforcing",
            "2": "Galvanized reinforcing",
            "3": "Other coated reinforcing",
            "4": "Cathodic protection",
            "6": "Polymer impregnated",
            "7": "Internally sealed",
            "8": "Unknown",
            "9": "Other",
            "0": "None",
            "N": "Not applicable (no deck)"
        }
    },
    "PERCENT_ADT_TRUCK_109": {
        "category": "Percentage of ADT that is truck traffic",
        "type": "numerical",
        "datatype": float,
        "unit": "percent",
        "code_map": None
    },
    "PERCENT_ADT_TRUCK_109_SEMANTIC": {
        "category": "Truck traffic percentage",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "PERCENT_ADT_TRUCK_109",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 0    },
            { "category": "very_low",    "min": 0,    "max": 5    },
            { "category": "low",         "min": 5,    "max": 10   },
            { "category": "moderate",    "min": 10,   "max": 20   },
            { "category": "high",        "min": 20,   "max": 40   },
            { "category": "very_high",   "min": 40,   "max": None }
        ],
        "code_map": {
            "none": "no truck traffic recorded",
            "very_low": "very low truck traffic share (<5%)",
            "low": "low truck traffic share (5–10%)",
            "moderate": "moderate truck traffic share (10–20%)",
            "high": "high truck traffic share (20–40%)",
            "very_high": "very high truck traffic share (>40%)"
        }
    },
    "NATIONAL_NETWORK_110": {
    "category": "National network for trucks designation",
    "type": "nominal",
    "datatype": int,
    "unit": None,
    "code_map": {
        "0": "Not part of the national network for trucks",
        "1": "Part of the national network for trucks"
    }
    },
    "PIER_PROTECTION_111": {
        "category": "Pier or abutment protection condition",
        "type": "nominal",
        "datatype": int,
        "unit": None,
        "code_map": {
            "1": "Navigation protection not required",
            "2": "Protection in place and functioning",
            "3": "Protection in place but deteriorated",
            "4": "Protection in place but re-evaluation suggested",
            "5": "None present but re-evaluation suggested"
        }
    },
    "SCOUR_CRITICAL_113": {
        "category": "Scour critical bridge evaluation",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "N": "Bridge not over waterway",
            "U": "Unknown foundation – not evaluated for scour (plan of action required)",
            "T": "Bridge over tidal waters – not evaluated but considered low risk",
            "9": "Foundations on dry land well above flood elevation",
            "8": "Foundations stable – scour above top of footing (resists scour or countermeasures in place)",
            "7": "Countermeasures installed – risk mitigated; plan of action implemented",
            "6": "Evaluation not yet made",
            "5": "Foundations stable – scour within limits of footing or piles",
            "4": "Foundations stable but action required to protect exposed foundations",
            "3": "Scour critical – unstable foundations for assessed/calculated conditions",
            "2": "Scour critical – extensive scour; foundations unstable by observation or evaluation",
            "1": "Scour critical – imminent pier/abutment failure, bridge closed to traffic",
            "0": "Scour critical – bridge failed and closed to traffic"
        }
    },
    "FUTURE_ADT_114": {
        "category": "Future average daily traffic forecast",
        "type": "numerical",
        "datatype": float,
        "unit": "vehicles per day",
        "code_map": None
    },
    "FUTURE_ADT_114_SEMANTIC": {
        "category": "Forecast average daily traffic",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "FUTURE_ADT_114",
        "bin_rules": [
            { "category": "none",           "min": None, "max": 1      },
            { "category": "low",            "min": 1,    "max": 20000  },
            { "category": "moderate",       "min": 20000,"max": 50000  },
            { "category": "high",           "min": 50000,"max": 100000 },
            { "category": "very_high",      "min": 100000,"max": 200000 },
            { "category": "extremely_high", "min": 200000,"max": None  }
        ],
        "code_map": {
            "none": "no forecast traffic available",
            "low": "low future traffic volume (<20k ADT)",
            "moderate": "moderate future traffic volume (20k–50k ADT)",
            "high": "high future traffic volume (50k–100k ADT)",
            "very_high": "very high future traffic volume (100k–200k ADT)",
            "extremely_high": "extremely high future traffic volume (>200k ADT)"
        }
    },
    "BRIDGE_CONDITION": {
        "category": "Overall bridge condition classification",
        "type": "nominal",
        "datatype": str,
        "unit": None,
        "code_map": {
            "G": "Good condition – lowest NBI rating ≥ 7",
            "F": "Fair condition – lowest NBI rating between 5 and 6",
            "P": "Poor condition – lowest NBI rating ≤ 4"
        }
    },
    "LOWEST_RATING": {
        "category": "Lowest NBI element rating",
        "type": "nominal",
        "datatype": int,
        "unit": "rating (0–9 scale)",
        "code_map": {
            "9": "Excellent condition",
            "8": "Very good condition",
            "7": "Good condition",
            "6": "Satisfactory condition",
            "5": "Fair condition",
            "4": "Poor condition",
            "3": "Serious condition",
            "2": "Critical condition",
            "1": "Imminent failure condition",
            "0": "Failed condition"
        }
    },
    "DECK_AREA": {
        "category": "Deck surface area",
        "type": "numerical",
        "datatype": float,
        "unit": "square meters",
        "code_map": None
    },
    "DECK_AREA_SEMANTIC": {
        "category": "Deck area",
        "type": "nominal_derived",
        "datatype": "string",
        "unit": None,
        "source": "DECK_AREA",
        "bin_rules": [
            { "category": "none",        "min": None, "max": 1     },
            { "category": "very_small",  "min": 1,    "max": 200   },
            { "category": "small",       "min": 200,  "max": 1000  },
            { "category": "medium",      "min": 1000, "max": 3000  },
            { "category": "large",       "min": 3000, "max": 7000  },
            { "category": "very_large",  "min": 7000, "max": None  }
        ],
        "code_map": {
            "none": "no defined deck area",
            "very_small": "a very small deck area (<200 m²)",
            "small": "a small deck area (200–1000 m²)",
            "medium": "a medium deck area (1000–3000 m²)",
            "large": "a large deck area (3000–7000 m²)",
            "very_large": "a very large deck area (>7000 m²)"
        }
    }
}

# Clean and Remove Irrelevant Columns

In [4]:
import pandas as pd

print("\n🧩 Loading and cleaning NBI feature dataset...")

# === 1️⃣ Load Features ===
# skiprows=3 keeps the same behavior as original script
full_f = pd.read_csv(NBIfeatures_path, skiprows=3)
print(f"✅ Loaded NBI features: {full_f.shape[0]:,} rows × {full_f.shape[1]:,} columns")

# === 3️⃣ Drop columns with all missing values ===
num_missing_drop = full_f.shape[1] - full_f.dropna(axis=1, how="all").shape[1]
if num_missing_drop > 0:
    print(f"🧹 Dropping {num_missing_drop} columns that are entirely NaN.")
    full_f = full_f.dropna(axis=1, how="all").copy()

# === 4️⃣ Optional missing-value filtering (commented out historically) ===
def print_null_percentages(df):
    """Utility to display percent of missing per column."""
    percent_missing = df.isnull().mean() * 100
    print("📊 Percentage of missing values per column:")
    print(percent_missing.round(2))
    return percent_missing

def remove_high_nan(df, threshold_percent=5):
    """Remove columns with more than threshold% NaN values."""
    percent_missing = print_null_percentages(df)
    cols_to_drop = percent_missing[percent_missing > threshold_percent].index
    if len(cols_to_drop) > 0:
        print(f"🧹 Removing {len(cols_to_drop)} columns with >{threshold_percent}% missing values.")
        df = df.drop(columns=cols_to_drop)
    return df

# Optional — uncomment to apply stricter NaN filtering
# full_f = remove_high_nan(full_f, threshold_percent=5)

# === 5️⃣ Remove columns with constant values ===
constant_cols = [c for c in full_f.columns if full_f[c].nunique(dropna=False) == 1 and c != "STATE_CODE_001"]
if constant_cols:
    print(f"🗑️ Dropping {len(constant_cols)} constant columns: {constant_cols}")
    full_f = full_f.drop(columns=constant_cols).copy()

# === 6️⃣ Optional uniqueness filter (commented out previously) ===
# threshold = 0.9 * len(full_f)
# high_unique = [c for c in full_f.columns if full_f[c].nunique() >= threshold]
# print(f"Columns with ≥90% unique values: {high_unique}")
# full_f = full_f.drop(columns=high_unique).copy()

# === 7️⃣ Drop irrelevant / reference / duplicate columns ===
reference_columns = [
    "In OP?", "In T2T?", "In National Network?", "STRUCTURE_NUMBER_008",
    # "Special Single Span?"  # optional
]
dup_columns = ["LAT_016", "LONG_017"]
not_rel_columns = ["Bridge #", 'Special Single Span?', 'ADT > (parameter)?', 'RECORD_TYPE_005A', 'DIRECTION_005E', 'PLACE_CODE_004', 'OWNER_022', 'CULVERT_COND_062', 'WORK_DONE_BY_075B', 'OTHR_STATE_STRUC_NO_099', 'BRIDGE_LEN_IND_112', 'FED_AGENCY', 'SUBMITTED_BY', 'BRIDGE_IMP_COST_094', 'FRACTURE_LAST_DATE_093A', 'IMP_LEN_MT_076', 'KILOPOINT_011', 'LRS_INV_ROUTE_013A', 'MIN_NAV_CLR_MT_116', 'OTHER_STATE_CODE_098A', 'OTHER_STATE_PCNT_098B', 'ROADWAY_IMP_COST_095', 'ROUTE_NUMBER_005D', 'SPEC_LAST_DATE_093C', 'SUBROUTE_NO_013B', 'TEMP_STRUCTURE_103', 'TOTAL_IMP_COST_096', 'UNDWATER_LAST_DATE_093B', 'YEAR_ADT_030', 'YEAR_OF_FUTURE_ADT_115', 'YEAR_OF_IMP_097']
# "OTHR_STATE_STRUC_NO_099", "MAINTENANCE_021", "OWNER_022" # optional

for label, cols in {
    "reference": reference_columns,
    "duplicate": dup_columns,
    "not_relevant": not_rel_columns,
}.items():
    # Exclude STATE_CODE_001 from being dropped
    existing = [c for c in cols if c in full_f.columns and c != "STATE_CODE_001"]
    if existing:
        print(f"🗑️ Dropping {len(existing)} {label} columns: {existing}")
        full_f = full_f.drop(columns=existing).copy()
    else:
        print(f"ℹ️ No {label} columns found to drop.")

# === 8️⃣ Remaining feature summary ===
print(f"\n📦 Remaining features: {full_f.shape[1]} columns")
missing_values = full_f.isna().sum()
columns_with_missing = missing_values[missing_values > 0]
if not columns_with_missing.empty:
    print("\n⚠️ Columns with missing values:")
    print(columns_with_missing)

# === 9️⃣ Rename columns for consistency ===
rename_map = {
    "Structure ID ": "STRUCTURE_ID",
    "Long. (Decimal Degrees)": "LONG",
    "Lat. (Decimal Degrees)": "LAT",
}
existing_renames = {k: v for k, v in rename_map.items() if k in full_f.columns}
missing_rename = [k for k in rename_map if k not in full_f.columns]
if missing_rename:
    print(f"⚠️ Skipping missing columns during rename: {missing_rename}")
full_f = full_f.rename(columns=existing_renames)
print("✅ Column renaming completed.")



🧩 Loading and cleaning NBI feature dataset...
✅ Loaded NBI features: 4,914 rows × 133 columns
🧹 Dropping 2 columns that are entirely NaN.
🗑️ Dropping 2 constant columns: ['RECORD_TYPE_005A', 'BRIDGE_LEN_IND_112']
🗑️ Dropping 4 reference columns: ['In OP?', 'In T2T?', 'In National Network?', 'STRUCTURE_NUMBER_008']
🗑️ Dropping 2 duplicate columns: ['LAT_016', 'LONG_017']
🗑️ Dropping 29 not_relevant columns: ['Bridge #', 'Special Single Span?', 'ADT > (parameter)?', 'DIRECTION_005E', 'PLACE_CODE_004', 'OWNER_022', 'CULVERT_COND_062', 'WORK_DONE_BY_075B', 'OTHR_STATE_STRUC_NO_099', 'FED_AGENCY', 'SUBMITTED_BY', 'BRIDGE_IMP_COST_094', 'FRACTURE_LAST_DATE_093A', 'IMP_LEN_MT_076', 'KILOPOINT_011', 'LRS_INV_ROUTE_013A', 'MIN_NAV_CLR_MT_116', 'OTHER_STATE_CODE_098A', 'OTHER_STATE_PCNT_098B', 'ROADWAY_IMP_COST_095', 'ROUTE_NUMBER_005D', 'SPEC_LAST_DATE_093C', 'SUBROUTE_NO_013B', 'TEMP_STRUCTURE_103', 'TOTAL_IMP_COST_096', 'UNDWATER_LAST_DATE_093B', 'YEAR_ADT_030', 'YEAR_OF_FUTURE_ADT_115', 'YE

# Display Cleaned NBI Dataset

In [5]:
full_f

,STRUCTURE_ID,LAT,LONG,STATE_CODE_001,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,FEATURES_DESC_006A,FACILITY_CARRIED_007,...,MEMBRANE_TYPE_108B,DECK_PROTECTION_108C,PERCENT_ADT_TRUCK_109,NATIONAL_NETWORK_110,PIER_PROTECTION_111,SCOUR_CRITICAL_113,FUTURE_ADT_114,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA
0,,47.985717,-122.227122,53,6,8,0,61,'EAST WATERWAY','SPRUANCE BLVD.',...,8,8,10.0,0,NaN,5,60,F,6,3273.76
1,,47.697975,-122.619500,53,6,8,0,35,'LAGOON','BUSHNELL DRIVE',...,0,0,0.0,0,NaN,5,20,F,6,105.56
2,,48.212150,-121.933131,53,6,8,0,61,'JIM CREEK','BLUE MOUNTAIN ROAD',...,0,0,8.0,0,1.0,5,25,F,5,123.82
3,,47.567592,-122.551703,53,6,8,0,35,'PATROL RD. & BEACH DRIVE','FUEL STORAGE FACIL',...,0,0,2.0,0,1.0,N,24,F,6,148.83
4,,47.769275,-122.707925,53,8,0,0,35,'CATTAIL LAKE DAM','AMBERJACK AVENUE',...,N,N,0.0,0,NaN,5,25,G,7,329.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,DAPFORLE,47.027881,-122.513600,53,8,0,0,53,'MUCK CREEK','PINCUS ROAD',...,N,N,0.0,0,NaN,8,10,G,7,77.35
4910,DAPFORLE,47.028931,-122.465500,53,8,0,0,53,'MUCK CREEK','30TH INF REGT RD',...,N,N,40.0,0,NaN,8,10,G,7,77.35
4911,DAPFORLE,47.116689,-122.500300,53,8,0,0,53,'E. LINCOLN/PERIMETER RD','JBLM ACCESS CORRID',...,0,1,5.0,0,NaN,N,18000,G,7,1387.56
4912,DAPFORLE,47.107989,-122.589800,53,8,0,0,53,'AMERICAN LAKE','NCO BEACH RD',...,N,N,20.0,0,NaN,8,500,G,7,63.44


# Make Coordinates Column


In [6]:
full_f["COORDINATES"] = list(zip(full_f["LAT"], full_f["LONG"]))

# Take Temporal Columns and Find the Difference with Current Year

In [7]:
import pandas as pd
import numpy as np
from datetime import date

# Get the current date object
today = date.today()

# Extract the year from the date object
CURRENT_YEAR = today.year

# Find all temporal columns
temporal_cols = [
    col for col, meta in modified_nbi_schema.items()
    if meta.get("type") == "temporal" and col in full_f.columns
]

for col in temporal_cols:
    diff_col = f"{col}_DIFF"

    def compute_diff(value):
        try:
            # Extract year: handles strings like '2012-05-14', '2012', timestamps, etc.
            year = pd.to_datetime(value, errors='coerce').year
            if pd.isna(year):
                return None
            return CURRENT_YEAR - int(year)
        except:
            return None

    full_f[diff_col] = full_f[col].apply(compute_diff)




# Check Columns in Schema and NBI Dataset


In [8]:
# columns in the DataFrame
df_cols = set(full_f.columns)

# keys in the dictionary
dict_entries = set(modified_nbi_schema.keys())

# intersections and exclusives
in_both = df_cols & dict_entries
only_in_df = df_cols - dict_entries
only_in_dict = dict_entries - df_cols

print("Columns present in BOTH full_f and modified_nbi_schema:")
print(sorted(in_both))

print("\nColumns ONLY in full_f:")
print(sorted(only_in_df))

print("\nEntries ONLY in modified_nbi_schema:")
print(sorted(only_in_dict))



Columns present in BOTH full_f and modified_nbi_schema:
['ADT_029', 'APPR_KIND_044A', 'APPR_RAIL_036C', 'APPR_RAIL_END_036D', 'APPR_ROAD_EVAL_072', 'APPR_SPANS_046', 'APPR_TYPE_044B', 'APPR_WIDTH_MT_032', 'BASE_HWY_NETWORK_012', 'BRIDGE_CONDITION', 'CHANNEL_COND_061', 'COORDINATES', 'COUNTY_CODE_003', 'DATE_OF_INSPECT_090', 'DATE_OF_INSPECT_090_DIFF', 'DECK_AREA', 'DECK_COND_058', 'DECK_GEOMETRY_EVAL_068', 'DECK_PROTECTION_108C', 'DECK_STRUCTURE_TYPE_107', 'DECK_WIDTH_MT_052', 'DEGREES_SKEW_034', 'DESIGN_LOAD_031', 'DETOUR_KILOS_019', 'FACILITY_CARRIED_007', 'FEATURES_DESC_006A', 'FEDERAL_LANDS_105', 'FRACTURE_092A', 'FUNCTIONAL_CLASS_026', 'FUTURE_ADT_114', 'HIGHWAY_DISTRICT_002', 'HIGHWAY_SYSTEM_104', 'HISTORY_037', 'HORR_CLR_MT_047', 'INSPECT_FREQ_MONTHS_091', 'INVENTORY_RATING_066', 'INV_RATING_METH_065', 'LAT', 'LAT_UND_MT_055B', 'LAT_UND_REF_055A', 'LEFT_CURB_MT_050A', 'LEFT_LAT_UND_MT_056', 'LOCATION_009', 'LONG', 'LOWEST_RATING', 'MAINTENANCE_021', 'MAIN_UNIT_SPANS_045', 'MAX_S

# Split Data Into Numerical, Natural Language, and Coordinates


In [9]:
import os
import pandas as pd

# --------------------------------------------------------------------
# OUTPUT DIRECTORY
# --------------------------------------------------------------------
out_dir = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
os.makedirs(out_dir, exist_ok=True)

# --------------------------------------------------------------------
# STEP 1 — Extract lists of column names by schema type
# --------------------------------------------------------------------
numerical_types = {"numerical", "numerical_coded"}
nominal_types = {"nominal", "nl"}

numerical_cols = [
    col for col, meta in modified_nbi_schema.items()
    if meta.get("type") in numerical_types and col in full_f.columns
]

nominal_cols = [
    col for col, meta in modified_nbi_schema.items()
    if meta.get("type") in nominal_types and col in full_f.columns
]

# Ensure STRUCTURE_ID is always included if present
if "STRUCTURE_ID" in full_f.columns:
    numerical_cols = ["STRUCTURE_ID"] + [c for c in numerical_cols if c != "STRUCTURE_ID"]
    nominal_cols   = ["STRUCTURE_ID"] + [c for c in nominal_cols if c != "STRUCTURE_ID"]


# --------------------------------------------------------------------
# STEP 2 — Create filtered DataFrames
# --------------------------------------------------------------------
df_numerical = full_f[numerical_cols]
df_nominal   = full_f[nominal_cols]
df_coords    = full_f[["STRUCTURE_ID", "COORDINATES"]]


# --------------------------------------------------------------------
# STEP 3 — Save to CSVs
# --------------------------------------------------------------------
df_numerical.to_csv(os.path.join(out_dir, "nbi_numerical.csv"), index=False)
df_nominal.to_csv(os.path.join(out_dir, "nbi_nominal.csv"), index=False)
df_coords.to_csv(os.path.join(out_dir, "structure_coordinates.csv"), index=False)

print("All 3 CSV files created successfully.")


All 3 CSV files created successfully.


# Check Typing in Schema


In [10]:
unique_types = {meta.get("type") for meta in modified_nbi_schema.values()}
print(unique_types)


{'numerical', 'temporal', 'nominal', 'nominal_derived', 'nl', 'reference', 'numerical_coded'}


# Create Nominal Dataset From Numerical Binnings

In [12]:
import os
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# Output directory
# ---------------------------------------------------------
out_dir = r"C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data"
os.makedirs(out_dir, exist_ok=True)

# ---------------------------------------------------------
# Identify semantic (nominal_derived) schema columns
# ---------------------------------------------------------
semantic_cols = {
    col: meta
    for col, meta in modified_nbi_schema.items()
    if meta.get("type") == "nominal_derived"
}

# ---------------------------------------------------------
# Function to apply bin rules to a value
# ---------------------------------------------------------
def apply_bin_rules(value, rules):
    try:
        v = float(value)
    except:
        return None  # invalid → null category

    for rule in rules:
        rmin = rule.get("min")
        rmax = rule.get("max")

        # Check lower bound
        if rmin is not None and v < rmin:
            continue

        # Check upper bound
        if rmax is not None and v >= rmax:
            continue

        # Inside this bin
        return rule["category"]

    return None  # if no bin matched


# ---------------------------------------------------------
# Build semantic DataFrame
# ---------------------------------------------------------
semantic_df = pd.DataFrame()

# Always include STRUCTURE_ID
if "STRUCTURE_ID" in full_f.columns:
    semantic_df["STRUCTURE_ID"] = full_f["STRUCTURE_ID"]

# Process each semantic column
for semantic_col, meta in semantic_cols.items():
    source_col = meta.get("source")
    bin_rules = meta.get("bin_rules", [])

    if source_col not in full_f.columns:
        print(f"Warning: source column '{source_col}' not found for {semantic_col}")
        semantic_df[semantic_col] = None
        continue

    semantic_df[semantic_col] = full_f[source_col].apply(
        lambda x: apply_bin_rules(x, bin_rules)
    )


# ---------------------------------------------------------
# Save the resulting semantic CSV
# ---------------------------------------------------------
out_path = os.path.join(out_dir, "nbi_semantic_nominal_derived.csv")
semantic_df.to_csv(out_path, index=False)

print("Semantic CSV created successfully:", out_path)


Semantic CSV created successfully: C:\Users\wongb\Bridge-ML\Bridge-ML-LLM-Embedding-Architecture\enriched_data\nbi_semantic_nominal_derived.csv
